# Shifting Db2 to a Db2 Container
Containerizing an existing Db2 database on an on-premises system to OpenShift, Kubernetes, or CP4D is the most common scenario for using Db2 Shift. To move a Db2 database to a POD you will require the following information:
* Source Database details
* Destination location
* Type of Containerization environment
* Shift Options
The target of the Db2 Shift operation can be OpenShift, a Kubernetes cluster, or Cloud Pak for Data.

## Terminal Window
Before running the examples in this lab, you will need to open up a terminal window. If you have not already done so, see the <a href="./Terminal.ipynb">Opening up a Terminal Window</a> section. 

The commands that you need to copy and execute in the terminal window are surrounded with a blue box. Select all the text in the box and then use the copy keys (^C or &#x2318;C) to place the text into the clipboard.

<div class="boxed">The command you need to copy and paste into the terminal window are found in here.
</div>

When you switch to the terminal window, you will use the appropriate Paste command (^&#x21E7;v or &#x2318;v). This will load the command into the terminal window and immediately execute it.

## Identify the Source Database
The Db2 database server is we are using in this lab is found on machine `10.0.0.1` (Server 1). This server has several databases available to do a shift from. The first step is to connect to Server 1 using by using a secure shell connection. The password for db2inst1 is **db2inst1**.

<div class="boxed">ssh db2inst1@10.0.0.1
</div>

While logged in as db2inst1@server1, use the command line to determine what databases are available.

<div class="boxed">db2 list db directory
</div>

You should see five databases:
* ONLINE – Flight on time arrivals database
* REPO – Data Management Console repository
* BANKING – Banking data for machine learning
* SAMPLE – Db2 SAMPLE database
* TRADING – Simulated stock trading database

You are going to move the **TRADING** database in this lab. 

Each chapter in this lab will use a different database. You can choose to use any of the databases in the list above, but the commands and results shown in the labs will be for the recommended database.

## Gather Target Database Details
You will need three pieces of information:
* Database name (`TRADING`)
* The project that the Db2U pod was deployed in (`db2u`)
* The POD name (`c-demo-db2u-0`)

To double check that you can access the Db2u pod, issue the following command.

<div class="boxed">kubectl get pods -n db2u
</div>

If the command is successful, you will see the list of services in the Db2U namespace.
```
NAME                                    READY   STATUS      RESTARTS   AGE
db2u-operator-manager-c849d76c4-pcd6d   1/1     Running     15         12d
c-demo-instdb-dddbc                     0/1     Completed   0          59m
c-demo-etcd-0                           1/1     Running     0          58m
c-demo-db2u-0                           1/1     Running     0          57m
c-demo-restore-morph-ps2b7              0/1     Completed   0          57m
```

## Gather Shift Details 
Before we jump into running Db2 Shift, we need to gather some details of the database we want to shift and the location. Here are some fields that you will need to have information on and what to look for in the Db2 Shift dialog.

* Source Database
  
  The source database is the name of the database that you want to move to the new location (`TRADING`). Note that you can have the same or different database name at the target. If you provide a different database name at the target, the program will copy the database from the source and place it on the target and use the existing name.
<p>
* Instance Owner
 
  The Db2 Shift program assumes that the current userid you are logged into is the owner of the instance (`db2inst1`). This is necessary due to the requirement to access the underlying files that are used by Db2. If you supply the source owner value, Db2 Shift will double check that you are working in the correct instance and the settings files are consistent.
<p>
* Target Client
 
  The pod client for a deploy must be supplied as part of the Db2 Shift command. In our lab we are shifting Db2 to a Kubernetes cluster so the Kubernetes option  must be selected (Kubernetes).
Note that you need to have the client (`Kubernetes`) installed locally or else the Db2 Shift program will be unable to communicate with the target cluster.
<p>
* Destination Database
 
  The destination database name can be the same as the source database, or a completely different name (`DB2OLTP`). Make sure that if you are creating a database with a different name that it doesn't currently exist on your target system. Otherwise, the target database will be deleted! Enter the name of the Db2 database that was created on the Kubernetes system.
<p>
* Destination Pod Namespace 
 
  In Kubernetes deployments, the location of a pod is associated with a namespace, while in OpenShift deployments, the pod is associated with a project (`db2u`). 
When the Db2 operator was deployed, `db2u` was used as the namespace for deploying Db2. This is the value that you would supply here.
<p>
* Destination Pod Name
 
  For deployments to OpenShift, Kubernetes, or CP4D, you must supply the name of the POD that Db2U is running in. The OpenShift or Kubernetes client should be used to check that the target namespace or project is available (`c-demo-db2u-0`).
<p>
* Move Options
 
  The MOVE option is set to "Database" which will take a copy of your Db2 database and copy it into the target Db2U container or instance. No other processing is done with the database.
<p>
* Performance: Threading
 
  The copy phase of the Db2 Shift program can use multiple threads to transmit data to a destination. This setting allows you to increase the parallelism up to 8 threads. As you increase the number of threads, the amount of data being transmitted increases, at the expense of greater CPU usage and network congestion. The default value is 4 which strikes a balance between overhead and network performance.
<p>
* Performance: Compression
 
  RSYNC compresses the data during the transfer process to allow for faster movement of data. The amount of compression can be adjusted from 0 to 9 with 0 turning off compression and values between 1 and 9 increasing the amount of compression applied to the data. Higher compression values will result in more CPU usage and may not significantly reduce the size of the data stream. 

  For high-speed networks, a value of 0 is recommended unless there is a requirement to reduce network traffic.  Set this value to zero for the lab.

## Start the Db2 Shift command
In the terminal window, change to the directory to `~/c2c`. This is the directory that the Db2 Shift command was installed in. Start the program without any parameters:

<div class="boxed">cd ~/c2c
./db2shift
</div>

You will be presented with the disclaimer screen.

![Disclaimer](images/c2c-disclaimer.jpg)
 
Press Enter to continue to the main screen.

![C2C Main](images/c2c-main.jpg)
 
The scenario that we are running in this first lab is at the top: Shift a Db2 database to OpenShift, Kubernetes, or CP4D. Make sure that the cursor is positioned on the first option and hit the Enter key. The Shift to Db2U panel will display.

![C2C Shift Pod](images/c2c-shiftpod.jpg)

Update the information on this screen with the values you collected.
* Database Name: `TRADING`
* Instance Owner: `db2inst1`
* Target: `Kubernetes`
* Target Database: `db2oltp`
* POD Project: `db2u`
* POD Name: `c-demo-db2u-0`
The following settings will not be used during this lab and should be set to the default values.
* Metadata: `Refresh`
* Sync: `None`
* Move Options: `Database`
* Exclude Routines: `Off`
* Database Mode: `Online`
* Thread Count: `4`
* Compression: `0`
* Overrides: `None`

## Running the Shift
Once you are finished entering all the details required, press the ^X (CTRL-X) key to review the command.
If the checks were successful, a summary of the command is displayed (**Note:** Your screen may be different).

![Summary](images/c2c-shiftsummary.jpg)
 
At this point you can run the Shift command or save it for future use. Press ^X again to run the command.
As the program begins the shift operation, it will display various progress messages (**Note:** Your screen may be different). When the shift completes, a summary screen will be displayed that should either be successful or indicate a problem with the shift.

![Final](images/c2c-success.jpg)

At this point you can check to see whether the target database has the contents of the source database in it. The first step is to use the Kubernetes client to shell into the Db2U container and check the contents of the database. Exit the Db2 Shift program by hitting `ESC` until you return to the command line.

<div class="boxed">clear
kubectl exec -i c-demo-db2u-0 -n db2u -- bash << EOF
cat << EOT > /tmp/db2cmd.db2
CONNECT TO DB2OLTP;
LIST TABLES FOR SCHEMA TRADING;
QUIT;
EOT
su - db2inst1 "db2 -tvf /tmp/db2cmd.db2"
EOF

## Summary
In this lab you:
* Used the kubectl command to check the status of the Db2U pod
* Examined the databases available for shifting into Db2U
* Learned how to use the Db2 Shift UI to shift a database to a pod
* Checked the contents of the shifted pod

#### Credits: 2024 George Baklarz [baklarz@ca.ibm.com]